In [1]:
import tushare as ts
import pandas as pd
import time
import datetime
from tqdm import tqdm
import re
import pprint
import json

In [ ]:
ts.set_token('53cd3b985c649c978160c6ec04bce24f4fbd2ebcb4673e8f2fba9a43')

pro = ts.pro_api()

In [ ]:
# 注意这里的股票列表应该从本地csv获取，避免出现新增股票
codes = []
with open('new_import/stock_codes.json', 'r') as f:
    js = json.load(f)
    for k, v in js.items():
        for s in v:
            s = s[2:] + '.' + s[:2]
            s = s.upper()
            codes.append(s)

In [ ]:
today = datetime.date.today()
today = today.strftime('%Y%m%d')

In [ ]:
pro.top10_holders(ts_code='000001.SZ', start_date='20200101', end_date=today).drop_duplicates().iloc[:10]

In [ ]:
holder_list = []
i = 0
for code in tqdm(codes):
    df = pro.top10_holders(ts_code=code, start_date='20200101', end_date=today).drop_duplicates().iloc[:10]
    holder_list.append(df)
    time.sleep(6)
    i += 1

In [ ]:
i

In [ ]:
relation_df = pd.concat(holder_list)

# 从这里开始

In [2]:
relation_df = pd.read_csv('relation.csv')

In [3]:
len(relation_df)

6700

In [4]:
relation_df.head()

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio
0,600075.SH,20201016,20200930,李慧琼,18364944.0,1.35
1,600075.SH,20201016,20200930,新疆天业(集团)有限公司,728351574.0,53.57
2,600075.SH,20201016,20200930,石河子市锦富国有资本投资运营有限公司,67760942.0,4.98
3,600075.SH,20201016,20200930,上海银叶投资有限公司-银叶攻玉5号私募证券投资基金,30508800.0,2.24
4,600075.SH,20201016,20200930,万家共赢-中信银行-万家共赢安泰1号专项资产管理计划,26874176.0,1.98


In [5]:
relation_df[relation_df['holder_name'] == '天津中冀万泰投资管理有限公司']

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio
4316,000657.SZ,20201029,20200930,天津中冀万泰投资管理有限公司,11764705.0,1.12


In [6]:
relation_df = relation_df.drop_duplicates(['ts_code', 'ann_date', 'holder_name'])

In [7]:
len(relation_df)

6685

In [8]:
def filter_func(x: pd.Series) -> str:
    s = x['holder_name']
    s = s.split('-')[0]  # 去掉横杠
    s = s.split('一')[0]
    s = re.sub('\\(.*\\)', '', s)  # 去掉空格和内容
    s = re.sub('股份|责任|有限|公司|集团|省|市', '', s)
    return s

In [9]:
relation_df['name'] = relation_df.apply(filter_func, axis=1)

In [10]:
relation_df.head()

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio,name
0,600075.SH,20201016,20200930,李慧琼,18364944.0,1.35,李慧琼
1,600075.SH,20201016,20200930,新疆天业(集团)有限公司,728351574.0,53.57,新疆天业
2,600075.SH,20201016,20200930,石河子市锦富国有资本投资运营有限公司,67760942.0,4.98,石河子锦富国有资本投资运营
3,600075.SH,20201016,20200930,上海银叶投资有限公司-银叶攻玉5号私募证券投资基金,30508800.0,2.24,上海银叶投资
4,600075.SH,20201016,20200930,万家共赢-中信银行-万家共赢安泰1号专项资产管理计划,26874176.0,1.98,万家共赢


In [11]:
len(set(relation_df['name']))

4863

In [12]:
len(set(relation_df['holder_name']))

5289

In [13]:
holder_df = relation_df['name'].to_frame()

In [14]:
holder_df = holder_df.drop_duplicates()
holder_df = holder_df.reset_index(drop=True)

In [15]:
len(holder_df)

4863

In [16]:
holder_df['holderId:ID(Holder)'] = holder_df.index

In [17]:
holder_df[holder_df['holderId:ID(Holder)'] == 7]

,name,holderId:ID(Holder)
7,金石期货,7


In [18]:
hold2id = {
    row['name']: row['holderId:ID(Holder)']
    for i, row in holder_df.iterrows()
}

In [19]:
relation_df[':START_ID(Holder)'] = relation_df.apply(lambda x: hold2id[x['name']], axis=1)

In [20]:
relation_df.head()

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio,name,:START_ID(Holder)
0,600075.SH,20201016,20200930,李慧琼,18364944.0,1.35,李慧琼,0
1,600075.SH,20201016,20200930,新疆天业(集团)有限公司,728351574.0,53.57,新疆天业,1
2,600075.SH,20201016,20200930,石河子市锦富国有资本投资运营有限公司,67760942.0,4.98,石河子锦富国有资本投资运营,2
3,600075.SH,20201016,20200930,上海银叶投资有限公司-银叶攻玉5号私募证券投资基金,30508800.0,2.24,上海银叶投资,3
4,600075.SH,20201016,20200930,万家共赢-中信银行-万家共赢安泰1号专项资产管理计划,26874176.0,1.98,万家共赢,4


In [21]:
relation_df = relation_df.drop(columns=['end_date', 'hold_amount', 'name'])

In [22]:
relation_df.head()

,ts_code,ann_date,holder_name,hold_ratio,:START_ID(Holder)
0,600075.SH,20201016,李慧琼,1.35,0
1,600075.SH,20201016,新疆天业(集团)有限公司,53.57,1
2,600075.SH,20201016,石河子市锦富国有资本投资运营有限公司,4.98,2
3,600075.SH,20201016,上海银叶投资有限公司-银叶攻玉5号私募证券投资基金,2.24,3
4,600075.SH,20201016,万家共赢-中信银行-万家共赢安泰1号专项资产管理计划,1.98,4


In [23]:
relation_df['ann_date'] = pd.to_datetime(relation_df['ann_date'], format='%Y%m%d')

In [24]:
relation_df.head()

,ts_code,ann_date,holder_name,hold_ratio,:START_ID(Holder)
0,600075.SH,2020-10-16,李慧琼,1.35,0
1,600075.SH,2020-10-16,新疆天业(集团)有限公司,53.57,1
2,600075.SH,2020-10-16,石河子市锦富国有资本投资运营有限公司,4.98,2
3,600075.SH,2020-10-16,上海银叶投资有限公司-银叶攻玉5号私募证券投资基金,2.24,3
4,600075.SH,2020-10-16,万家共赢-中信银行-万家共赢安泰1号专项资产管理计划,1.98,4


In [25]:
relation_df.columns = [':END_ID(Company)', 'annDate:date','holderFullname:string', 'holdRatio:double', ':START_ID(Holder)']

In [26]:
holder_df[':LABEL'] = 'Holder'
relation_df[':TYPE'] = 'hold_stock'

In [27]:
holder_df.head()

,name,holderId:ID(Holder),:LABEL
0,李慧琼,0,Holder
1,新疆天业,1,Holder
2,石河子锦富国有资本投资运营,2,Holder
3,上海银叶投资,3,Holder
4,万家共赢,4,Holder


In [28]:
relation_df.head()

,:END_ID(Company),annDate:date,holderFullname:string,holdRatio:double,:START_ID(Holder),:TYPE
0,600075.SH,2020-10-16,李慧琼,1.35,0,hold_stock
1,600075.SH,2020-10-16,新疆天业(集团)有限公司,53.57,1,hold_stock
2,600075.SH,2020-10-16,石河子市锦富国有资本投资运营有限公司,4.98,2,hold_stock
3,600075.SH,2020-10-16,上海银叶投资有限公司-银叶攻玉5号私募证券投资基金,2.24,3,hold_stock
4,600075.SH,2020-10-16,万家共赢-中信银行-万家共赢安泰1号专项资产管理计划,1.98,4,hold_stock


In [29]:
holder_df.to_csv('new_import/holder.csv', index=False)
relation_df.to_csv('new_import/holder_company.csv', index=False)